
<br>
<font>
<!-- <img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" alt="SUT logo" width=300 height=300 align=left class="saturate"> -->
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=200 height=200>
<br>
<font color=0F5298 size=7>
Machine Learning <br>
<font color=2565AE size=5>
Electrical Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 4 <br>
<font color=696880 size=4>
<!-- <br> -->


____

# Personal Data

In [1]:
student_number = '400104686'
first_name = 'Parsa'
last_name = 'Yousefpoor'

# Introduction

In this assignment, we will be performing clustering on Spotify songs.

# Data Preprocessing

In the next cell, import the libraries you'll need.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import os

from google.colab import drive
drive.mount('/content/drive')
os.chdir("drive/My Drive/IML_4_2")

Mounted at /content/drive


In the `spotify.csv` file, load the data. Exclude unrelated features and retain only the track name and the features you believe are relevant.

In [4]:
usecols = ['track_id','track_name', 'track_artist', 'track_album_id','track_album_name',
                'track_album_release_date', 'playlist_name', 'playlist_id']
df = pd.read_csv('spotify.csv', usecols = usecols)

In this cell, you should implement a standard scalar function from scratch and applying it to your data. Explian importance behind using a standard scalar and the potential complications that could arise in clustering if it's not employed. (you can't use `sklearn.preprocessing.StandardScaler` but you are free to use `sklearn.preprocessing.LabelEncoder`)

**Answer:**

A Standard Scaler is crucial in data preprocessing, especially for machine learning algorithms that are sensitive to the scale of the data, such as PCA, LDA, and k-means clustering. Standard scaling ensures that each feature has a mean of 0 and a standard deviation of 1. This uniformity prevents features with larger scales from dominating the distance calculations and model training.

**Complications Without Standard Scaling:**

1. Bias in Distance-Based Algorithms: Algorithms like k-means clustering rely on distance metrics. Without scaling, features with larger ranges can disproportionately influence the clustering results.
2. Convergence Issues: Gradient-based algorithms might converge more slowly or to suboptimal solutions if the feature scales vary significantly.
3. Model Interpretability: Inconsistent feature scales can complicate the interpretation of model coefficients and predictions.

In [5]:
def standard_scaler(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    scaled_X = (X - mean) / std
    return scaled_X

numeric_features = df.select_dtypes(include=[np.number]).columns
df_scaled = df.copy()
df_scaled[numeric_features] = standard_scaler(df[numeric_features])
print(df_scaled.head())
print("Means of scaled features:\n", df_scaled[numeric_features].mean())
print("\nStandard deviations of scaled features:\n", df_scaled[numeric_features].std())

                 track_id                                         track_name  \
0  6f807x0ima9a1j3VPbc7VN  I Don't Care (with Justin Bieber) - Loud Luxur...   
1  0r7CVbZTWZgbTCYdfa2P31                    Memories - Dillon Francis Remix   
2  1z1Hg7Vb0AhHDiEmnDE79l                    All the Time - Don Diablo Remix   
3  75FpbthrwQmzHlBJLuGdC7                  Call You Mine - Keanu Silva Remix   
4  1e8PAfcKUYoKkxPhrHqw4x            Someone You Loved - Future Humans Remix   

       track_artist          track_album_id  \
0        Ed Sheeran  2oCs0DGTsRO98Gh5ZSl2Cx   
1          Maroon 5  63rPSO264uRjW1X5E6cWv6   
2      Zara Larsson  1HoSmj2eLcsrR0vE9gThr4   
3  The Chainsmokers  1nqYsOef1yKKuGOVchbsk6   
4     Lewis Capaldi  7m7vv9wlQ4i0LFuJiE2zsQ   

                                    track_album_name track_album_release_date  \
0  I Don't Care (with Justin Bieber) [Loud Luxury...               2019-06-14   
1                    Memories (Dillon Francis Remix)               2019-12

# Dimensionality Reduction

One method for dimensionality reduction is Principal Component Analysis (PCA). Use its implementation from the `sklearn` library to reduce the dimensions of your data. Then, by using an appropriate cut-off for the `_explained_variance_ratio_` in the PCA algorithm, determine the number of principal components to retain.

In [ ]:
# TODO: Write your code here

# Clustering

Implement K-means for clustering from scratch.

In [ ]:
# TODO: Write your code here

Using the function you've created to execute the K-means algorithm eight times on your data, with the number of clusters ranging from 2 to 9. For each run, display the genre of each cluster using the first two principal components in a plot.

In [ ]:
# TODO: Write your code here

The Silhouette score and the Within-Cluster Sum of Squares (WSS) score are two metrics used to assess the quality of your clustering. You can find more information about these two methods [here](https://medium.com/analytics-vidhya/how-to-determine-the-optimal-k-for-k-means-708505d204eb). Plot the Silhouette score and the WSS score for varying numbers of clusters, and use these plots to determine the optimal number of clusters (k).

In [ ]:
# TODO: Write your code here

# Checking Output

To see how good was our clustering we will use a sample check and t-SNE method.

first randomly select two song from every cluster and see how close these two songs are.

In [ ]:
# TODO: Write your code here

Using t-SNE reduce dimension of data pointe to 2D and plot it to check how good datapoints are clustered (implementing this part is optional and have extra points)

In [ ]:
# TODO: Write your code here